In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import regex as re
from time import sleep as sl

In [5]:
def eh_pessoa(url):
    html = urlopen(url)
    site = html.read()
    soup = BeautifulSoup(site, 'html.parser')
    
    infobox = soup.find_all("table", class_=re.compile("^[iI]nfobox"))
    if infobox is not None:
        atributos = []
        for linha in infobox:
            x = linha.find_all('td', {'scope': "row"})
            for i in x:
                atributos.append(i)
    else:
        print("Sem infobox")
    
    for at in atributos:
        # Criar if's com regex para encontrar atributos de pessoas
        # Ex: Nome completo, Nome, Data de nascimento, Nascimento, Nacionalidade, Assinatura etc.
        if re.search("[nN]ome", at.text):
            return True
        
    return False

In [8]:
def find_url(url):
    """Produtor: pega links e coloca na fila"""
    wikipedia_base_url = 'https://pt.wikipedia.org'
    html = urlopen(url)
    site = html.read()
    soup = BeautifulSoup(site, "html.parser")
    
    todos_os_links_tags = soup.find_all('a', href=re.compile(r'^/wiki/'))
    todos_os_hrefs = {tag['href'] for tag in todos_os_links_tags}

    # Regex: começa com /wiki/ e não contém :, (), [], dígitos
    padrao_regex = re.compile(r"^/wiki/(?!.*[:()\[\]0-9.!])(.*_.*)$")
    
    links_mantidos_hrefs = {
        href for href in todos_os_hrefs if padrao_regex.match(href)
    }
    
    links_to_search = [f'{wikipedia_base_url}{link}' for link in links_mantidos_hrefs]

    # joga na fila
    for link in links_to_search:
        print(f"[Produtor] Adicionando link: {link}")
        link_queue.put(link)
        sl(0.1) 
